## Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit

In [2]:
from datetime import datetime
from FireSimulator import *
from FireSimulatorUtilities import *
import glob
import itertools
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import os
import pickle
import time

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

## Utility functions

In [3]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, w = x.size() 
        return x.view(N, -1)

In [14]:
def heuristic_trajectory(pos, center, num_actions, img_st, isfire):
    trajectory = []
    trajectory.append((pos[0],pos[1]))
    img_dim = img_st.shape[0]
    
    if isfire:
        fires_r, fires_c = np.where(img_st==1)
        neighbors = [(-1,0),(1,0),(0,1),(0,-1)]
        fire = []
        bdry = []
        for r,c in zip(fires_r,fires_c):
            counter = 0
            for (dr,dc) in neighbors:
                rn = r + dr
                cn = c + dc
                if rn>=0 and rn<img_dim and cn>=0 and cn<img_dim and img_st[rn,cn] == 0:
                    counter += 1
                    
            x = col_to_x(c) - img_dim//2 + pos[0]
            y = row_to_y(img_dim,r) - img_dim//2 + pos[1]
            if counter >= 2:
                bdry.append((x,y))
            else:
                fire.append((x,y))
                
        using_bdry = False
        if len(bdry) > 0:
            dists = [(np.abs(x-pos[0])+np.abs(y-pos[1]),(x,y)) for (x,y) in bdry]
            using_bdry = True
        else:
            dists = [(np.abs(x-pos[0])+np.abs(y-pos[1]),(x,y)) for (x,y) in fire]
            
        target = min(dists)[1]
        while len(trajectory) < num_actions+1:
            loc = trajectory[-1]
            if loc == target:
                if using_bdry:
                    bdry.remove(target)
                elif len(fire) > 0:
                    fire.remove(target)
                    
                using_bdry = False
                if len(bdry) > 0:
                    dists = [(np.abs(x-loc[0])+np.abs(y-loc[1]),(x,y)) for (x,y) in bdry]
                    target = min(dists)[1]
                    using_bdry = True
                elif len(fire) > 0:
                    dists = [(np.abs(x-loc[0])+np.abs(y-loc[1]),(x,y)) for (x,y) in fire]
                    target = min(dists)[1]
                else:
                    target = trajectory[-1]
                    
            dists = []
            for a in range(9):
                new_loc = actions_to_trajectory(trajectory[-1],[a])[1]
                dists.append((np.abs(new_loc[0]-target[0])+np.abs(new_loc[1]-target[1]),new_loc))
                
            trajectory.append(min(dists)[1])
            #print(trajectory)
        
    else:
        for k in range(num_actions):
            dists = []
            for a in range(9):
                new_loc = actions_to_trajectory(trajectory[-1],[a])[1]
                dists.append((np.abs(center-new_loc[0])+np.abs(center-new_loc[1]),new_loc))
            
            trajectory.append(min(dists)[1])
            
    return trajectory

## Load data

In [5]:
# directory = os.path.join(os.getcwd(), 'data')

# pattern = os.path.join(directory,'states_seed_*')

# data = {}
# k = 0
# total_states = 0
# for file in glob.glob(pattern):
#     fh = open(file, 'rb')
#     sub_data = pickle.load(fh)
#     data[k] = sub_data
#     fh.close()
#     k += 1
#     total_states += sub_data.shape[0]
    
# print('loaded %d sims from file' %(k))
# print('for a total of %d states' %(total_states))

## Network datatype [cpu/gpu]

In [6]:
torch.cuda.is_available()

True

In [7]:
# dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor

## Build the network

In [8]:
class eelfff_net(nn.Module):
    """
    network to approximate Q function
    """
    def __init__(self, act_seq=4, img_dim=8):
        self.act_seq = act_seq
        self.img_dim = img_dim
        self.num_poss_actions = 9
        
        C, H, W = 1, img_dim, img_dim
        hidden_dim = 2048
        
        # conv layer settings
        nf1 = 128; nf2 = 128
        fs1 = 2; fs2 = 2
        cv_s1 = 1; cv_s2 = 1
        cv_p1 = 0; cv_p2 = 0
        
        # pool layer settings
        p_sz1 = 4; p_sz2 = 2
        p_st1 = 1; p_st2 = 1
        
        # calculate affine layer size
        Hp1 = 1 + (H + 2*cv_p1 - fs1) // cv_s1
        Wp1 = Hp1
        Hpp1 = 1 + (Hp1 - p_sz1) // p_st1
        Wpp1 = Hpp1
        
        Hp2 = 1 + (Hpp1 + 2*cv_p2 - fs2) // cv_s2
        Wp2 = Hp2
        Hpp2 = 1 + (Hp2 - p_sz2) // p_st2
        Wpp2 = Hpp2
        
        #aff_flat_size = nf2*Hpp2*Wpp2 + 2*act_seq
        aff_flat_size = nf1*Hpp1*Wpp1 + 2*(act_seq+1)
        
        super(eelfff_net, self).__init__()
        # cnn structure
        self.cnn = nn.Sequential(
                        nn.Conv2d(C, nf1, kernel_size=fs1, stride=cv_s1, padding=cv_p1),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(p_sz1,stride=p_st1),
                        #nn.Conv2d(nf1, nf2, kernel_size=fs2, stride=cv_s2, padding=cv_p2),
                        #nn.ReLU(inplace=True),
                        #nn.MaxPool2d(p_sz2,stride=p_st2),
                        Flatten()
                    )
        
        # nonlinear structure
        self.aff = nn.Sequential(
                        nn.Linear(aff_flat_size, hidden_dim),
                        nn.ReLU(inplace=True),
                        nn.Linear(hidden_dim, self.num_poss_actions*act_seq)
                    )
        
    def forward(self, img, act):
        img_exp = img.unsqueeze(0)
        img_exp = img_exp.unsqueeze(0)
        act_exp = act.unsqueeze(0)
        feat = self.cnn(img_exp)
        feat = torch.cat((feat, act_exp), dim=1)
        Q = self.aff(feat)
        
        return Q.view(N,self.num_poss_actions,self.act_seq)

test implementation of network with random data

In [9]:
tic = time.clock()
N = 1
img_dim = 8
act_seq = 6
model = eelfff_net(act_seq, img_dim).type(dtype)

img = torch.randn(img_dim,img_dim).type(dtype)
act = torch.randn(2*(act_seq+1)).type(dtype)

img_var = Variable(img)
act_var = Variable(act)

Q = model(img_var, act_var)
toc = time.clock()

print(Q.size())
print("%0.2fs = %0.2fm elapsed for this test" %(toc-tic,(toc-tic)/60))

torch.Size([1, 9, 6])
462.12s = 7.70m elapsed for this test


## Define a reward function

In [16]:
def eelfff_reward(states, trajs, fire_flags, other_trajs):
    N = len(states)
    grid_size = states[0].shape[0]
    center = math.ceil(grid_size/2)
    neighbors = [(-1,0),(0,-1),(1,0),(0,1)]
    #reward = Variable(torch.zeros(1), requires_grad=True).type(dtype)
    reward = 0
    
    for n in range(N):
        st = states[n]
        traj = trajs[n]
        other_traj = other_trajs[n]
        has_fires = fire_flags[n]
        
        # reward for treating fires and boundary fires
        # that weren't already treated by the agent
        if has_fires:
            treated = []
            for (x,y) in traj:
                r = y_to_row(grid_size,y)
                c = x_to_col(x)

                # reward for treating a fire
                if r>=0 and r<grid_size and c>=0 and c<grid_size and st[r,c] == 1 and (x,y) not in treated:
                    reward += 1

                    counter = 0 
                    for (dc,dr) in neighbors:
                        rn = r + dr
                        cn = c + dc
                        if rn>=0 and rn<grid_size and cn>=0 and cn<grid_size and st[rn,cn] == 0:
                            counter += 1

                    # bonus for treating a boundary fire
                    if counter >= 2:
                        reward += 4
                        
                    treated.append((x,y))
           
        # reward for approaching center [if no fires in image]
        else:
            for k in range(len(traj)-1):
                x1, y1 = traj[k]
                x2, y2 = traj[k+1]
                if np.abs(x2-center)+np.abs(y2-center) < np.abs(x1-center)+np.abs(y1-center):
                    reward += 1.0/(len(traj)-1)
    
        # penalty for intersecting with 'nearest agent'
        if not set(traj).isdisjoint(other_traj):
            reward += -2*len(set(traj).intersection(other_traj))            
    
    return reward/N

test reward function with random data

In [11]:
tic = time.clock()

states = np.zeros((3,5,5)).astype(np.uint8)
states[:,2,2] = 1
trajs = []
trajs.append([(5,5),(5,5),(4,4)])
trajs.append([(3,3),(3,3),(3,3)])
trajs.append([(5,5),(4,4),(3,3)])
other_trajs = []
other_trajs.append([(1,1),(1,2),(1,1)])
other_trajs.append([(1,1),(1,2),(1,3)])
other_trajs.append([(1,1),(2,2),(3,3)])
fire_flags = [False, True, True]

reward = eelfff_reward(states, trajs, fire_flags, other_trajs)
print('minibatch reward: %0.2f' %reward)

toc = time.clock()
print("%0.2fs = %0.2fm elapsed for this test" %(toc-tic,(toc-tic)/60))

minibatch reward: 2.83
0.00s = 0.00m elapsed for this test


## Train the network

In [38]:
# simulator and network parameters
grid_size = 50
num_agents = [2,5,10]
D = []
memory_size = 1000000
dp = 0.15/0.2763
act_seq = 6
img_dim = 8
center = math.ceil(grid_size/2)

# agent initialization parameters
spawn_loc = np.arange(grid_size//3//2,grid_size,grid_size//3,1)
perturbs = np.arange(-grid_size//3//2+1,grid_size//3//2+1,1)

# create network instance
model = eelfff_net(act_seq=act_seq, img_dim=img_dim).type(dtype)
model.train()

# optimizer and its parameters
gamma = 0.9
epsilon = 0.05
batch_size = 32
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

# loss function
loss_fn = torch.nn.MSELoss()

In [41]:
seeds = range(1)

In [88]:
tic = time.clock()
# run simulator many times
for s in seeds:
    np.random.seed(s)
    
    # initialize simulator
    sim = FireSimulator(grid_size, rng=s)
    
    # initialize agent positions
    n = np.squeeze(np.random.choice(num_agents, 1))
    agent_pos = np.random.choice(spawn_loc, (n,2)) + np.random.choice(perturbs, (n,2))
    agent_pos = agent_pos.astype(np.int32)
        
    # run to termination
    while not sim.end:
        
        control = []
        new_agent_pos = np.zeros((n,2)).astype(np.int32)
        agent_data = {}
        
        # generate control for each agent
        for k in range(n):
            agent_data[k] = {}
            
            # generate and save image
            img, img_st, isfire = CreateImageBW(sim.state, agent_pos[k,:])
            agent_data[k]['img'] = img

            # find nearest neighbor and their trajectory, and save it
            dists = [(np.linalg.norm(agent_pos[k,:]-pos,1),i) for i,pos in enumerate(agent_pos) if i != k]
            min_idx = min(dists)[1]
            other_img, other_img_st, other_isfire = CreateImageBW(sim.state, agent_pos[min_idx,:])
            other_traj = heuristic_trajectory(agent_pos[min_idx,:], center, act_seq, other_img_st, other_isfire)
            agent_data[k]['other_traj'] = other_traj
            
            # epsilon-greedy action choice: either random or use network
            if np.random.sample() <= epsilon:
                actions = np.random.random_integers(0, high=8, size=(act_seq,))
            else:
                img_var = Variable(torch.from_numpy(img)).type(dtype)
                other_traj_var = Variable(torch.from_numpy(np.asarray(other_traj).reshape((-1,)))).type(dtype)
                Q = model(img_var, other_traj_var)[0]
                actions = np.argmax(Q.data.cpu().numpy(),axis=0)
                
            agent_data[k]['actions'] = actions 
            
            # generate trajectory and generate control from trajectory
            traj = actions_to_trajectory(agent_pos[k,:], actions)
            control.extend(FindGridIntersections(sim.state, traj))
            
            # calculate and store reward for agent
            reward = eelfff_reward([sim.state], [traj], [isfire], [other_traj])
            agent_data[k]['reward'] = reward
                        
            # store agent's new position
            new_agent_pos[k,:] = [traj[-1][0], traj[-1][1]]
            
        # remove duplicates from control sequence
        control = list(set(control))
        #if control:
        #    print('control is not empty')
        #    print(control)
        #    5/0
        
        # step simulator
        sim.step(control, dbeta=dp)
        
        # update agent position
        agent_pos = new_agent_pos
        
        # grab new state information and add to replay memory
        isterminal = False
        for k in range(n):
            # generate and save image
            img, img_st, isfire = CreateImageBW(sim.state, agent_pos[k,:])
            agent_data[k]['next_img'] = img
            
            # find nearest neighbor and their trajectory
            dists = [(np.linalg.norm(agent_pos[k,:]-pos,1),i) for i,pos in enumerate(agent_pos) if i != k]
            min_idx = min(dists)[1]
            other_img, other_img_st, other_isfire = CreateImageBW(sim.state, agent_pos[min_idx,:])
            other_traj = heuristic_trajectory(agent_pos[min_idx,:], center, act_seq, other_img_st, other_isfire)
            agent_data[k]['next_other_traj'] = other_traj
        
            # check for terminal state
            if sim.end:
                isterminal = True
                
            D.append((agent_data[k]['img'],agent_data[k]['other_traj'],agent_data[k]['actions'],
                      agent_data[k]['reward'],agent_data[k]['next_img'],
                      agent_data[k]['next_other_traj'],isterminal))
                
        # create minibatch from replay memory
        loss = 0
        if len(D) <= batch_size:
            batch = D            
        else:
            batch = []
            idxs = np.random.random_integers(0,high=len(D)-1,size=batch_size)
            for k in range(batch_size):
                batch.append(D[idxs[k]])

        # calculate loss over batch
        # exp indices: 0-img, 1-other_traj, 2-actions, 3-reward, 4-next_img, 5-next_other_traj
        for exp in batch:
            isterminal = exp[-1]
            tt = None
            if isterminal:
                tt = exp[3]
            else:
                next_img_var = Variable(torch.from_numpy(exp[4])).type(dtype)
                next_other_traj_var = Variable(torch.from_numpy(np.asarray(exp[5]).reshape((-1,)).astype(np.int32))).type(dtype)

                #Q = model(next_img_var, next_other_traj_var)[0]
                #maxQ = Q.max(dim=0)[0].mean()
                Q = model(next_img_var, next_other_traj_var)[0].data.cpu().numpy()
                maxQ = np.mean(np.amax(Q, axis=0))
                tt = exp[3] + gamma*maxQ

            tt = Variable(tt*torch.ones(1), requires_grad=False).type(dtype)

            img_var = Variable(torch.from_numpy(exp[0])).type(dtype)
            other_traj_var = Variable(torch.from_numpy(np.asarray(exp[1]).reshape((-1,)))).type(dtype)
            Q = model(img_var, other_traj_var)[0]
            x = Q[torch.from_numpy(exp[2]).type(torch.cuda.LongTensor)].diag().mean()
            loss += loss_fn(x,tt)

        loss /= batch_size

        # update network
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # drop from memory if too many elements
        if len(D) > memory_size:
            D = D[len(D)-memory_size:]
            
    print(sim.stats,sim.stats[0]/np.sum(sim.stats))
        
toc = time.clock()
print("%0.2fs = %0.2fm elapsed" %(toc-tic,(toc-tic)/60))

[   9    0 2491] 0.0036
[  12    0 2488] 0.0048
[   9    0 2491] 0.0036
[  14    0 2486] 0.0056
[  14    0 2486] 0.0056
[  21    0 2479] 0.0084
[  15    0 2485] 0.006
[  30    0 2470] 0.012
[  11    0 2489] 0.0044
[   7    0 2493] 0.0028
160.97s = 2.68m elapsed


In [90]:
agent_pos

array([[ 0, 50],
       [13, 31],
       [ 1, 31],
       [45,  0],
       [43,  0],
       [ 2, 30],
       [18,  0],
       [24, 32],
       [17,  0],
       [ 8, -1]], dtype=int32)

In [87]:
type(np.asarray(other_traj)[0][0])

numpy.int64

In [82]:
type(np.asarray(exp[5])[0][0])

numpy.int8

In [75]:
type(exp[5][0][1])

numpy.int8

In [65]:
type(traj[0][0])

numpy.int32

In [64]:
torch.cuda.FloatTensor(traj)

RuntimeError: tried to construct a tensor from a nested float sequence, but found an item of type numpy.int32 at index (0, 0)

In [54]:
np.asarray(exp[5])

array([[15, 11],
       [15, 11],
       [15, 11],
       [15, 11],
       [15, 11],
       [15, 11],
       [15, 11]], dtype=int8)

In [49]:
np.asarray(other_traj).reshape((-1,))

array([15, 11, 15, 11, 15, 11, 15, 11, 15, 11, 15, 11, 15, 11], dtype=int8)

In [45]:
other_traj

[(15, 11), (15, 11), (15, 11), (15, 11), (15, 11), (15, 11), (15, 11)]

In [36]:
agent_pos.astype(np.int8)

array([[26, 25],
       [ 6, 16],
       [29,  6],
       [32, 39],
       [13, 20],
       [27, 34],
       [40, 20],
       [ 2, 34],
       [13,  7],
       [ 6, 44]], dtype=int8)

In [28]:
control

[]

In [29]:
len(D)

10000

In [ ]:
exp[2]

In [ ]:
Q[torch.from_numpy(exp[2]).type(torch.cuda.LongTensor)]

In [ ]:
Q[torch.from_numpy(exp[2]).type(torch.cuda.LongTensor)].diag()